## Unit 15. DNN(MLP) 課堂作業
- 課程編號: CHEXXXX
- 學年度: 114下
- 上課時間: 每週四 09:00-12:00
-
- 指導教授: ＯＯＯ 教授
- 學生姓名: ＯＯＯ
- 學號: m12345678
- email帳號: fcu.m12345678@gmail.com

---
### 環境設定

In [1]:
from pathlib import Path
import tensorflow as tf
import os

# ========================================
# 路徑設定 (兼容 Colab 與 Local)
# ========================================
UNIT_OUTPUT_DIR = 'P4_Unit15_HW'
SOURCE_DATA_DIR = 'HW'

try:
  from google.colab import drive
  IN_COLAB = True
  print("✓ 偵測到 Colab 環境，準備掛載 Google Drive...")
  drive.mount('/content/drive', force_remount=True)
except ImportError:
  IN_COLAB = False
  print("✓ 偵測到 Local 環境")
try:
  shortcut_path = '/content/CHE-AI-COURSE'
  os.remove(shortcut_path)
except FileNotFoundError:
  pass

if IN_COLAB:
  source_path = Path('/content/drive/My Drive/Colab Notebooks/CHE-AI-COURSE')
  os.symlink(source_path, shortcut_path)
  shortcut_path = Path(shortcut_path)
  if source_path.exists():
    NOTEBOOK_DIR = shortcut_path / 'Part_4'
    OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
    DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
    MODEL_DIR = OUTPUT_DIR / 'models'
    FIG_DIR = OUTPUT_DIR / 'figs'
  else:
    print(f"⚠️ 找不到路徑雲端CHE-AI-COURSE路徑，請確認自己的雲端資料夾是否正確")
  
else:
  NOTEBOOK_DIR = Path.cwd()
  OUTPUT_DIR = NOTEBOOK_DIR / 'outputs' / UNIT_OUTPUT_DIR
  DATA_DIR = NOTEBOOK_DIR / 'data' / SOURCE_DATA_DIR
  MODEL_DIR = OUTPUT_DIR / 'models'
  FIG_DIR = OUTPUT_DIR / 'figs'

NOTEBOOK_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n✓ Notebook工作目錄: {NOTEBOOK_DIR}")
print(f"✓ 數據來源目錄: {DATA_DIR}")
print(f"✓ 結果輸出目錄: {OUTPUT_DIR}")
print(f"✓ 模型輸出目錄: {MODEL_DIR}")
print(f"✓ 圖檔輸出目錄: {FIG_DIR}")


# ========================================
# 檢查 GPU 狀態
# ========================================
print(f"\nTensorFlow Version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"✓ 偵測到 GPU：{gpus[0].name}")
    print("  （訓練速度將明顯快於僅用 CPU）")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("△ 未偵測到 GPU。")
    print("  訓練速度將使用 CPU（速度較慢但仍可完成）")

✓ 偵測到 Local 環境

✓ Notebook工作目錄: d:\MyGit\CHE-AI-COURSE\Part_4
✓ 數據來源目錄: d:\MyGit\CHE-AI-COURSE\Part_4\data\HW
✓ 結果輸出目錄: d:\MyGit\CHE-AI-COURSE\Part_4\outputs\P4_Unit15_HW
✓ 模型輸出目錄: d:\MyGit\CHE-AI-COURSE\Part_4\outputs\P4_Unit15_HW\models
✓ 圖檔輸出目錄: d:\MyGit\CHE-AI-COURSE\Part_4\outputs\P4_Unit15_HW\figs

TensorFlow Version: 2.10.0
✓ 偵測到 GPU：/physical_device:GPU:0
  （訓練速度將明顯快於僅用 CPU）


---
### **I. 練習題: 建立一個DNN回歸模型**
 - 使用下面化工製程模擬數據集。

In [ ]:
import numpy as np
import pandas as pd

# 設定隨機種子
np.random.seed(42)

# 生成特徵數據
n_samples = 2000
data = {
    '進料流量': np.random.uniform(10, 50, n_samples),
    '反應器溫度': np.random.uniform(150, 250, n_samples),
    '反應器壓力': np.random.uniform(5, 15, n_samples),
    '催化劑濃度': np.random.uniform(0.5, 5, n_samples),
    '攪拌速度': np.random.uniform(100, 500, n_samples),
    '冷卻水溫度': np.random.uniform(15, 35, n_samples),
    '原料純度': np.random.uniform(90, 99.5, n_samples),
    '停留時間': np.random.uniform(30, 120, n_samples)
}

df = pd.DataFrame(data)

# 生成目標變數(產品轉化率) - 複雜非線性關係
conversion_rate = (
    0.3 * df['反應器溫度'] +
    0.2 * df['催化劑濃度'] * df['停留時間'] +
    -0.15 * (df['進料流量'] - 30)**2 +
    0.1 * np.log(df['攪拌速度']) * df['反應器壓力'] +
    0.05 * df['原料純度'] * df['反應器溫度'] / 100 +
    np.random.normal(0, 5, n_samples)  # 添加噪音
)/2

# 限制轉化率在合理範圍
conversion_rate = np.clip(conversion_rate, 0, 100)

# 模型預測目標
df['產品轉化率'] = conversion_rate

# 保存數據
df.to_csv(DATA_DIR/'reactor_data.csv', index=False, encoding='utf-8-sig')
print(f"數據集已生成: {DATA_DIR}/reactor_data.csv")
print(f"數據形狀: {df.shape}")
print(f"\n前5筆數據:\n{df.head()}")

### **1. 數據準備與探索**
- 載入數據並檢查基本統計資訊
- 繪製目標變數分布圖
- 繪製至少3個特徵與目標變數的散點圖
- 數據分割:訓練集70%、驗證集15%、測試集15%
- **對X和Y都進行StandardScaler標準化**

### **2. 模型建立**
- 使用Sequential API建立DNN模型
- 加入隱藏層, 使用ReLU激活函數
- 加入Dropout層
- 使用`model.summary()`查看模型結構
- 使用`plot_model()`繪製模型架構圖

### **3. 模型編譯**
- 使用Adam優化器
- 設定損失函數
- 評估指標
- 說明為何選擇這些設定

### **4. 模型訓練**
- 設定至少2個callbacks:
  - EarlyStopping
  - ModelCheckpoint
- 設定相關參數
- 使用validation_data進行驗證
- (可選)加入TensorBoard callback

### **5. 訓練過程視覺化**
- 繪製訓練與驗證的Loss曲線
- 繪製訓練與驗證的MAE曲線
- 分析是否有過擬合或欠擬合現象

### **6. 模型評估與預測**
- **重要**: 對預測結果進行反標準化
- 在測試集上計算MAE、RMSE、R²
- 繪製實際值vs預測值散點圖
- 繪製殘差圖
- 分析模型性能

### 7. 保存模型


---
## II. 額外加分作業 (自由繳交)
- 學生可自由選擇是否繳交加分作業. (下週上課前完成email電子檔)
- 分數會加在最後總成績上, 每份作業加0.1 ~ 1.0分.
- 分數加的不多, 真的不一定要交加分作業, 正常出席上課做好期末報告即可.
- 加分作業不一定要全部都做完, 想繳交至少完成其中一項實驗即可.
- 請務必自行完成! 你可以問AI, 問同學, 問學長姊, 甚至參考以前別人的作業, 但一定要自行[吸收][執行][整理]結果!
- 不要貼AI的答案寄給老師看, 你自己看就好.
- 如果系統自動比對發現作業內容(與他人雷同, 原創性低, 抄襲比例過高), 作業的分數會0分.
- 如果你直接100%複製別人的作業繳交, 你會直接被當掉(提供作業給別人複製的也一樣).
- 老師看你作業要花時間, 真的有心想寫加分作業, 請好好自己做. 

---
### 使用課堂作業所提供的化工製程模擬數據集
- 嘗試修改生成模擬數據的程式
- 改變參數, 改變公式, 添加特徵 ... 等等.
- 產生獨一無二屬於你的數據集
- 建立 DNN 模型來預測產品品質指標
- 然後開始進行下面實驗

### **實驗1: 網路深度影響**

比較以下4種網路架構:
- **淺層網路**: 2層 [64, 32]
- **中層網路**: 3層 [128, 64, 32]
- **深層網路**: 4層 [256, 128, 64, 32]
- **自行設計**:

**要求**:
- 其他參數保持一致
- 記錄每個模型的:
  - 訓練時間
  - 最佳驗證Loss
  - 測試集MAE、RMSE、R²
  - 總參數量(從model.summary()獲取)
- 製作比較表格
- **分析**: 哪種深度最適合?為什麼?

### **實驗2: Dropout Rate影響**

比較以下4種dropout rate:
- 0.0 (無dropout)
- 0.2
- 0.3
- 0.5

**要求**:
- 使用相同的網路架構
- 記錄訓練與驗證Loss的差距
- 分析過擬合程度
- **分析**: 最佳dropout rate是多少?

### **實驗3: Batch Size影響**

比較以下4種batch size:
- 16
- 32
- 64
- 128

**要求**:
- 記錄每個epoch的訓練時間
- 記錄最終測試集性能
- 計算每個epoch的iterations數量
- **分析**: batch size如何影響訓練速度和模型性能?

### **實驗4: 學習率影響**

比較以下4種"**固定**"學習率:
- 0.0001
- 0.001
- 0.01
- 0.1

**要求**:
- 觀察訓練曲線的收斂速度
- 記錄是否出現訓練不穩定
- **分析**: 最佳學習率是多少?學習率過大或過小會有什麼問題?
- 比較使用學習率自調諧衰減策略

### 💭 思考題

1. 為什麼在回歸任務中,Y數據也需要標準化?
2. 如果不進行反標準化直接計算評估指標,會有什麼問題?
3. 為什麼深層網路不一定比淺層網路好?
4. Dropout如何防止過擬合?它的工作原理是什麼?
5. 學習率過大和過小分別會導致什麼問題?
6. 在工業應用中,如何選擇合適的batch size?
7. 如果測試集性能遠差於驗證集,可能是什麼原因?

---
# 想對老師說的話